# CS 542 - TPCH Query Optimization - Group 2

In [1]:
#import libraries
import datetime
import time

import pandas as pd

In [2]:
#format and calculate dates
def calc_date(date, **kwargs):
    d = datetime.datetime.fromisoformat(date) + datetime.timedelta(**kwargs)
    d = d.isoformat().split('T')[0]
    return d



In [3]:
#return results from query execution
class Result:
    def __init__(self, rows):
        self.rows = rows

    def fetchall(self):
        return self.rows

In [4]:
#base query to calculate start and end times, and return results
class BQuery:
    template = """"""

    def render(self):
        return self.template

    def run(self, curr):
        query_string = self.render()
        start_time = time.time()
        results = curr.execute(query_string)
        end_time = time.time()
        run_time = end_time - start_time
        return results, run_time

In [9]:
#classes with all queries

class Q1(BQuery):
    template = """select l_returnflag, l_linestatus, sum(l_quantity) as sum_qty,
                     sum(l_extendedprice) as sum_base_price, sum(l_extendedprice*(1-l_discount)) as sum_disc_price,
                     sum(l_extendedprice*(1-l_discount)*(1+l_tax)) as sum_charge, avg(l_quantity) as avg_qty,
                    avg(l_extendedprice) as avg_price, avg(l_discount) as avg_disc, count(*) as count_order
                    from lineitem
                    where l_shipdate <= date('{date}') 
                    group by l_returnflag, l_linestatus
                    order by l_returnflag,l_linestatus;"""

    def __init__(self, days=-90):
        self.days = days

    def render(self):
        date = calc_date('1998-12-01', days=self.days)
        query = self.template.format(date=date)
        return query
    

class Q2(BQuery):
    template = """
                select s_acctbal, s_name, n_name, p_partkey, p_mfgr, s_address, s_phone, s_comment
                from part, supplier, partsupp, nation, region
                where p_partkey = ps_partkey and s_suppkey = ps_suppkey and p_size = 15 and
                p_type like '%brass' and s_nationkey = n_nationkey and n_regionkey = r_regionkey and
                r_name = 'europe' and
                ps_supplycost = (select min(ps_supplycost) from partsupp, supplier, nation, region
                 where p_partkey = ps_partkey and s_suppkey = ps_suppkey
                 and s_nationkey = n_nationkey and n_regionkey = r_regionkey and r_name = 'europe')
                order by s_acctbal, n_name, s_name, p_partkey
                limit 100;
                """  
    
class Q3(BQuery):
    template = """
                select l_orderkey, sum(l_extendedprice*(1-l_discount)) as revenue, o_orderdate, o_shippriority
                from customer, orders, lineitem
                where c_mktsegment = 'building' and c_custkey = o_custkey and l_orderkey = o_orderkey and
                o_orderdate < '1995-03-15' and l_shipdate > '1995-03-15'
                group by l_orderkey, o_orderdate, o_shippriority
                order by revenue desc, o_orderdate
                limit 10;
                """


class Q4(BQuery):
    template = """
                select o_orderpriority, count(*) as order_count from orders
                where o_orderdate >= '1993-07-01' and o_orderdate < '{date}'
                and exists (select * from lineitem where l_orderkey = o_orderkey and l_commitdate < l_receiptdate)
                group by o_orderpriority
                order by o_orderpriority;
                """

    def render(self):
        
        return self.template.format(date='1993-10-01')


class Q5(BQuery):
    template = """
                select n_name, sum(l_extendedprice*(1-l_discount)) as revenue
                from customer, orders, lineitem, supplier, nation, region
                where c_custkey = o_custkey and l_orderkey = o_orderkey and l_suppkey = s_suppkey
                and c_nationkey = s_nationkey and s_nationkey = n_nationkey and n_regionkey = r_regionkey
                and r_name = 'asia' and o_orderdate >= '1994-01-01'
                and o_orderdate < '{date}'
                group by n_name
                order by revenue desc;
                """

    def render(self):
        # add 1 year to '1994-01-01'
        date = '1995-01-01'
        return self.template.format(date=date)


class Q6(BQuery):
    template = """
                select sum(l_extendedprice * l_discount) as revenue
                from lineitem
                where l_shipdate >= date('1994-01-01') and l_shipdate < date('1994-01-01', '+1 year')
                and l_discount between .06 - 0.01 and .06 + 0.01 and l_quantity < 24;
                """


class Q7(BQuery):
    template = """
                select supp_nation, cust_nation, l_year, sum(volume) as revenue
                from ( select n1.n_name as supp_nation, n2.n_name as cust_nation, strftime('%y', l_shipdate) as l_year,
                 l_extendedprice*(1-l_discount) as volume
                 from supplier, lineitem, orders, customer, nation n1, nation n2
                 where s_suppkey = l_suppkey and o_orderkey = l_orderkey and c_custkey = o_custkey
                 and s_nationkey = n1.n_nationkey and c_nationkey = n2.n_nationkey and
                 ((n1.n_name = 'france' and n2.n_name = 'germany') or
                 (n1.n_name = 'germany' and n2.n_name = 'france')) and
                 l_shipdate between '1995-01-01' and '1996-12-31' ) as shipping
                group by supp_nation, cust_nation, l_year
                order by supp_nation, cust_nation, l_year;
                """


class Q8(BQuery):
    template = """
                select o_year, sum(case when nation = 'brazil' then volume else 0 end)/sum(volume) as mkt_share
                from (select strftime('%y', o_orderdate) as o_year, l_extendedprice*(1-l_discount) as volume, n2.n_name as nation
                 from part, supplier, lineitem, orders, customer, nation n1, nation n2, region
                 where p_partkey = l_partkey and s_suppkey = l_suppkey and l_orderkey = o_orderkey
                 and o_custkey = c_custkey and c_nationkey = n1.n_nationkey and
                 n1.n_regionkey = r_regionkey and r_name = 'america' and s_nationkey = n2.n_nationkey
                 and o_orderdate between '1995-01-01' and '1996-12-31' and p_type= 'economy anodized steel') as all_nations
                group by o_year
                order by o_year;
                """


class Q9(BQuery):
    template = """SELECT NATION, O_YEAR, SUM(AMOUNT) AS SUM_PROFIT
                FROM (SELECT N_NAME AS NATION, strftime('%Y', O_ORDERDATE) AS O_YEAR,
                 L_EXTENDEDPRICE*(1-L_DISCOUNT)-PS_SUPPLYCOST*L_QUANTITY AS AMOUNT
                 FROM PART, SUPPLIER, LINEITEM, PARTSUPP, ORDERS, NATION
                 WHERE S_SUPPKEY = L_SUPPKEY AND PS_SUPPKEY= L_SUPPKEY AND PS_PARTKEY = L_PARTKEY AND
                 P_PARTKEY= L_PARTKEY AND O_ORDERKEY = L_ORDERKEY AND S_NATIONKEY = N_NATIONKEY AND
                 P_NAME LIKE '%%green%%') AS PROFIT
                GROUP BY NATION, O_YEAR
                ORDER BY NATION, O_YEAR DESC;
                """


class Q10(BQuery):
    template = """
                SELECT C_CUSTKEY, C_NAME, SUM(L_EXTENDEDPRICE*(1-L_DISCOUNT)) AS REVENUE, C_ACCTBAL,
                N_NAME, C_ADDRESS, C_PHONE, C_COMMENT
                FROM CUSTOMER, ORDERS, LINEITEM, NATION
                WHERE C_CUSTKEY = O_CUSTKEY AND L_ORDERKEY = O_ORDERKEY AND O_ORDERDATE>= '1993-10-01' AND
                O_ORDERDATE < '{date}' AND
                L_RETURNFLAG = 'R' AND C_NATIONKEY = N_NATIONKEY
                GROUP BY C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT
                ORDER BY REVENUE DESC LIMIT 20;
                """

    def render(self):
        
        return self.template.format(date='1994-01-01')


class Q11(BQuery):
    template = """SELECT PS_PARTKEY, SUM(PS_SUPPLYCOST*PS_AVAILQTY) AS VALUE
                FROM PARTSUPP, SUPPLIER, NATION
                WHERE PS_SUPPKEY = S_SUPPKEY AND S_NATIONKEY = N_NATIONKEY AND N_NAME = 'GERMANY'
                GROUP BY PS_PARTKEY
                HAVING SUM(PS_SUPPLYCOST*PS_AVAILQTY) > (SELECT SUM(PS_SUPPLYCOST*PS_AVAILQTY) * 0.0001000000
                 FROM PARTSUPP, SUPPLIER, NATION
                 WHERE PS_SUPPKEY = S_SUPPKEY AND S_NATIONKEY = N_NATIONKEY AND N_NAME = 'GERMANY')
                ORDER BY VALUE DESC;"""

    def render(self):
        return self.template


class Q12(BQuery):
    template = """SELECT L_SHIPMODE,
                SUM(CASE WHEN O_ORDERPRIORITY = '1-URGENT' OR O_ORDERPRIORITY = '2-HIGH' THEN 1 ELSE 0 END) AS HIGH_LINE_COUNT,
                SUM(CASE WHEN O_ORDERPRIORITY <> '1-URGENT' AND O_ORDERPRIORITY <> '2-HIGH' THEN 1 ELSE 0 END ) AS LOW_LINE_COUNT
                FROM ORDERS, LINEITEM
                WHERE O_ORDERKEY = L_ORDERKEY AND L_SHIPMODE IN ('MAIL','SHIP')
                AND L_COMMITDATE < L_RECEIPTDATE AND L_SHIPDATE < L_COMMITDATE AND L_RECEIPTDATE >= DATE('1994-01-01')
                AND L_RECEIPTDATE < DATE('1994-01-01', '+1 year')
                GROUP BY L_SHIPMODE
                ORDER BY L_SHIPMODE;"""



class Q13(BQuery):
    template = """
                SELECT C_COUNT, COUNT(*) as CUSTDIST
                from (SELECT C_CUSTKEY, COUNT(O_ORDERKEY) as C_COUNT
                        from CUSTOMER LEFT OUTER JOIN ORDERS ON C_CUSTKEY = O_CUSTKEY
                            AND O_COMMENT NOT LIKE '%special%requests%'
                                GROUP BY C_CUSTKEY) AS C_ORDERS
                            GROUP BY C_COUNT
                            ORDER BY CUSTDIST DESC, C_COUNT DESC;
                """


class Q14(BQuery):
    template = """SELECT 100.00* SUM(CASE WHEN P_TYPE LIKE 'PROMO%%' THEN L_EXTENDEDPRICE*(1-L_DISCOUNT)
                ELSE 0 END) / SUM(L_EXTENDEDPRICE*(1-L_DISCOUNT)) AS PROMO_REVENUE
                FROM LINEITEM, PART
                WHERE L_PARTKEY = P_PARTKEY AND L_SHIPDATE >= '1995-09-01' AND L_SHIPDATE < '{date}';"""

    def render(self):
        #date'1995-09-01' + 1 month
        return self.template.format(date= '1995-10-01')


class Q15(BQuery):
    view_template = """
            create view revenue0 (supplier_no, total_revenue) as
            select
                l_suppkey,
                sum(l_extendedprice * (1 - l_discount))
            from lineitem
            where
                l_shipdate >= date('1996-01-01')
                and l_shipdate < date('1996-01-01', '+3 month')
            group by l_suppkey;"""

    template = """
            select s_suppkey, s_name, s_address, s_phone, total_revenue
                from supplier, revenue0
                where s_suppkey = supplier_no and total_revenue = ( select max(total_revenue) from revenue0)
                order by s_suppkey;"""

    def __init__(self, view_name='revenue0'):
        self.view_name = view_name

    def render_view(self):
        # add 3 months to '1996-01-01'
        date = '1996-04-01'
        return self.view_template #.format(name=self.view_name, date=date)

    def render(self):
        return self.template

    def run(self, curr):
        query_string = self.render()
        view_query = self.render_view()

        start_time = time.time()
        try:
            view_result = curr.execute(view_query)
        except:
            curr.execute(f'DROP VIEW {self.view_name}')
            raise Exception('View exists.')

        result = curr.execute(query_string)
        results = result.fetchall()
        result = Result(results)
        end_time = time.time()
        curr.execute(f'DROP VIEW {self.view_name}')
        run_time = end_time - start_time
        return result, run_time



class Q16(BQuery):
    template = """
                SELECT P_BRAND, P_TYPE, P_SIZE, COUNT(DISTINCT PS_SUPPKEY) AS SUPPLIER_CNT
                FROM PARTSUPP, PART
                WHERE P_PARTKEY = PS_PARTKEY AND P_BRAND <> 'Brand#45' AND P_TYPE NOT LIKE 'MEDIUM POLISHED%%'
                AND P_SIZE IN (49, 14, 23, 45, 19, 3, 36, 9) AND PS_SUPPKEY NOT IN (SELECT S_SUPPKEY FROM SUPPLIER
                 WHERE S_COMMENT LIKE '%%Customer%%Complaints%%')
                GROUP BY P_BRAND, P_TYPE, P_SIZE
                ORDER BY SUPPLIER_CNT DESC, P_BRAND, P_TYPE, P_SIZE;"""


class Q17(BQuery):
    template = """
                SELECT SUM(L_EXTENDEDPRICE)/7.0 AS AVG_YEARLY FROM LINEITEM, PART
                WHERE P_PARTKEY = L_PARTKEY AND P_BRAND = 'Brand#23' AND P_CONTAINER = 'MED BOX'
                AND L_QUANTITY < (SELECT 0.2*AVG(L_QUANTITY) FROM LINEITEM WHERE L_PARTKEY = P_PARTKEY);"""

    
class Q18(BQuery):
    template = """
            SELECT C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, SUM(L_QUANTITY)
            FROM CUSTOMER, ORDERS, LINEITEM
            WHERE O_ORDERKEY IN (SELECT L_ORDERKEY FROM LINEITEM GROUP BY L_ORDERKEY HAVING
             SUM(L_QUANTITY) > 300) AND C_CUSTKEY = O_CUSTKEY AND O_ORDERKEY = L_ORDERKEY
            GROUP BY C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE
            ORDER BY O_TOTALPRICE DESC, O_ORDERDATE
            LIMIT 100;"""


class Q19(BQuery):
    template = """
            SELECT SUM(L_EXTENDEDPRICE* (1 - L_DISCOUNT)) AS REVENUE
            FROM LINEITEM, PART
            WHERE (P_PARTKEY = L_PARTKEY AND P_BRAND = 'Brand#12' AND P_CONTAINER IN ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG') AND L_QUANTITY >= 1 AND L_QUANTITY <= 1 + 10 AND P_SIZE BETWEEN 1 AND 5
            AND L_SHIPMODE IN ('AIR', 'AIR REG') AND L_SHIPINSTRUCT = 'DELIVER IN PERSON')
            OR (P_PARTKEY = L_PARTKEY AND P_BRAND ='Brand#23' AND P_CONTAINER IN ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK') AND L_QUANTITY >=10 AND L_QUANTITY <=10 + 10 AND P_SIZE BETWEEN 1 AND 10
            AND L_SHIPMODE IN ('AIR', 'AIR REG') AND L_SHIPINSTRUCT = 'DELIVER IN PERSON')
            OR (P_PARTKEY = L_PARTKEY AND P_BRAND = 'Brand#34' AND P_CONTAINER IN ( 'LG CASE', 'LG BOX', 'LG PACK', 'LG PKG') AND L_QUANTITY >=20 AND L_QUANTITY <= 20 + 10 AND P_SIZE BETWEEN 1 AND 15
            AND L_SHIPMODE IN ('AIR', 'AIR REG') AND L_SHIPINSTRUCT = 'DELIVER IN PERSON');"""


class Q20(BQuery):
    template = """
                SELECT S_NAME, S_ADDRESS FROM SUPPLIER, NATION
                WHERE S_SUPPKEY IN (SELECT PS_SUPPKEY FROM PARTSUPP
                 WHERE PS_PARTKEY IN (SELECT P_PARTKEY FROM PART WHERE P_NAME like 'forest%%') AND
                 PS_AVAILQTY > (SELECT 0.5*sum(L_QUANTITY) FROM LINEITEM WHERE L_PARTKEY = PS_PARTKEY AND
                  L_SUPPKEY = PS_SUPPKEY AND L_SHIPDATE >= '1994-01-01' AND
                  L_SHIPDATE < '{date}')) AND S_NATIONKEY = N_NATIONKEY AND N_NAME = 'CANADA'
                ORDER BY S_NAME;"""

    def render(self):
        # add 1 year to '1994-01-01'
        date = '1995-01-01'
        return self.template.format(date=date)


class Q21(BQuery):
    template = """
    SELECT S_NAME, COUNT(*) AS NUMWAIT
    FROM SUPPLIER, LINEITEM L1, ORDERS, NATION WHERE S_SUPPKEY = L1.L_SUPPKEY AND
    O_ORDERKEY = L1.L_ORDERKEY AND O_ORDERSTATUS = 'F' AND L1.L_RECEIPTDATE> L1.L_COMMITDATE
    AND EXISTS (SELECT * FROM LINEITEM L2 WHERE L2.L_ORDERKEY = L1.L_ORDERKEY
     AND L2.L_SUPPKEY <> L1.L_SUPPKEY) AND
    NOT EXISTS (SELECT * FROM LINEITEM L3 WHERE L3.L_ORDERKEY = L1.L_ORDERKEY AND
     L3.L_SUPPKEY <> L1.L_SUPPKEY AND L3.L_RECEIPTDATE > L3.L_COMMITDATE) AND
    S_NATIONKEY = N_NATIONKEY AND N_NAME = 'SAUDI ARABIA'
    GROUP BY S_NAME
    ORDER BY NUMWAIT DESC, S_NAME
    LIMIT 100;"""


class Q22(BQuery):
    template = """
    SELECT CNTRYCODE, COUNT(*) AS NUMCUST, SUM(C_ACCTBAL) AS TOTACCTBAL
    FROM (SELECT SUBSTRING(C_PHONE,1,2) AS CNTRYCODE, C_ACCTBAL
     FROM CUSTOMER WHERE SUBSTRING(C_PHONE,1,2) IN ('13', '31', '23', '29', '30', '18', '17') AND
     C_ACCTBAL > (SELECT AVG(C_ACCTBAL) FROM CUSTOMER WHERE C_ACCTBAL > 0.00 AND
      SUBSTRING(C_PHONE,1,2) IN ('13', '31', '23', '29', '30', '18', '17')) AND
     NOT EXISTS ( SELECT * FROM ORDERS WHERE O_CUSTKEY = C_CUSTKEY)) AS CUSTSALE
    GROUP BY CNTRYCODE
    ORDER BY CNTRYCODE;"""





In [10]:
queries = {
    'Query 1': Q1(days=-90),
    'Query 2': Q2(),
    'Query 3': Q3(),
    'Query 4': Q4(),
    'Query 5': Q5(),
    'Query 6': Q6(),
    'Query 7': Q7(),
    'Query 8': Q8(),
    'Query 9': Q9(),
    'Query 10': Q10(),
    'Query 11': Q11(),
    'Query 12': Q12(),
    'Query 13': Q13(),
    'Query 14': Q14(),
    'Query 15': Q15(),
    'Query 16': Q16(),
    'Query 17': Q17(),
    'Query 18': Q18(),
    'Query 19': Q19(),
    'Query 20': Q20(),
    'Query 21': Q21(),
    'Query 22': Q22()
}

In [12]:
#import libraries
import csv
import sqlite3

import pandas as pd


In [13]:
# connect to the dataset
con = sqlite3.connect('C:/sqlite/tpch.db')
curr = con.cursor()

In [14]:
# save the records in individual files
def save_records(name, records):
    with open(f'data/query_results/{name}.csv', 'w') as f:
        csv_writer = csv.writer(f)
        for row in records:
            csv_writer.writerow(row)


In [ ]:
#loop through records and append runtime to list, save the list after loop finishes
data = []
futures = []
for name, query in queries.items():
    try:
        print("Running query ", name)
        result, run_time = query.run(curr)
    except Exception as e:
        print('Error with query', name, [i for i in result.fetchall()])
        print(e.with_traceback())
        continue
    records = [i for i in result.fetchall()]
    data.append({'query_name': name, 'num_records': len(records), 'time': run_time })
    print("Run time was", run_time)
    save_records(name, records)



df = pd.DataFrame(data)
df.to_csv('query_times_finals.csv', index = False)


# Optimized Query 20

In [18]:

class Q20Optim(BQuery):
    template = """select distinct s_name, s_address
                from supplier 
                    join nation on s_nationkey = n_nationkey
                    join (SELECT distinct PS_SUPPKEY, ps_partkey, ps_availqty 
                            FROM PARTSUPP
                                     WHERE PS_PARTKEY IN 
                                         (SELECT P_PARTKEY FROM PART WHERE P_NAME like 'forest%%')) on ps_suppkey = s_suppkey
                    join (select distinct L_PARTKEY, L_SUPPKEY, (0.5 * sum(l_quantity)) as l_qty
                                    from lineitem  
                                         WHERE l_shipdate >= date('1994-01-01')
                                            and l_shipdate < date('1994-01-01', '+1 year') 
                                            group by L_PARTKEY, L_SUPPKEY) on l_partkey = ps_partkey and l_suppkey = ps_suppkey
                where n_name = 'CANADA'
                        and ps_availqty > l_qty
                group by s_name, s_address
                order by s_name;"""

In [19]:
queries = {
    'Query 20 Optimized': Q20Optim()}

In [21]:

data = []
futures = []
for name, query in queries.items():
    #try:
    print("Running query ", name)
    result, run_time = query.run(curr)
    #except Exception as e:
    #    print('Error with query', name, [i for i in result.fetchall()])
    #    print(e.with_traceback())
    #    continue
    records = [i for i in result.fetchall()]
    data.append({'query_name': name, 'num_records': len(records), 'time': run_time })
    print("Run time was", run_time)
    save_records(name, records)



df = pd.DataFrame(data)

Running query  Query 20 Optimized
Run time was 4.444279432296753


In [22]:
df

,query_name,num_records,time
0,Query 20 Optimized,186,4.444279
